In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
url="https://books.toscrape.com/"
response=requests.get(url)
response

<Response [200]>

In [6]:
content=response.content
soup=BeautifulSoup(content,"html.parser")
soup

<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link href="s

In [14]:
#  get the books titles
def get_book_titles(con):
    Book_title=con.find_all('h3')
    Book_titles=[]
    for tags in Book_title:
        Book_titles.append(tags.text)
    return Book_titles

In [15]:
get_book_titles(soup)

['A Light in the ...',
 'Tipping the Velvet',
 'Soumission',
 'Sharp Objects',
 'Sapiens: A Brief History ...',
 'The Requiem Red',
 'The Dirty Little Secrets ...',
 'The Coming Woman: A ...',
 'The Boys in the ...',
 'The Black Maria',
 'Starving Hearts (Triangular Trade ...',
 "Shakespeare's Sonnets",
 'Set Me Free',
 "Scott Pilgrim's Precious Little ...",
 'Rip it Up and ...',
 'Our Band Could Be ...',
 'Olio',
 'Mesaerion: The Best Science ...',
 'Libertarianism for Beginners',
 "It's Only the Himalayas"]

In [46]:
#get_book_price
def get_book_price(con):
    Book_price_tags=con.find_all('p',class_='price_color')
    Book_price=[]
    for tag in Book_price_tags:
        Book_price.append(tag.text.replace('Â',''))
    return Book_price

In [47]:
get_book_price(soup)

['£51.77',
 '£53.74',
 '£50.10',
 '£47.82',
 '£54.23',
 '£22.65',
 '£33.34',
 '£17.93',
 '£22.60',
 '£52.15',
 '£13.99',
 '£20.66',
 '£17.46',
 '£52.29',
 '£35.02',
 '£57.25',
 '£23.88',
 '£37.59',
 '£51.33',
 '£45.17']

In [28]:
# get availability
def get_availability(con):
    Book_stock_tags=con.find_all('p',class_='instock availability')
    Book_stock = []
    for tag in Book_stock_tags:
        Book_stock.append(tag.text.strip())
    return Book_stock

In [29]:
get_availability(soup)

['In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock',
 'In stock']

In [32]:
#get link for each book
def Book_link(con):
    Book_link=[]
    for article in con:
        for link in article.find_all('a',href=True):
            url=link['href']
            links = 'https://books.toscrape.com/' + url
            if links not in Book_link:
                Book_link.append(links)
    return Book_link

In [36]:
Book_title_tags = soup.find_all('h3')
Book_link(Book_title_tags)

['https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html',
 'https://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html',
 'https://books.toscrape.com/catalogue/soumission_998/index.html',
 'https://books.toscrape.com/catalogue/sharp-objects_997/index.html',
 'https://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html',
 'https://books.toscrape.com/catalogue/the-requiem-red_995/index.html',
 'https://books.toscrape.com/catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
 'https://books.toscrape.com/catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
 'https://books.toscrape.com/catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html',
 'https://books.toscrape.com/catalogue/the-black-maria_991/index.html',
 'https://books.toscrape.com/catalogue/starving-hearts-triangular-trade-tr

In [37]:
#Collect data from multiple pages 
def get_con(url):
    response=requests.get(url)
    doc=BeautifulSoup(response.text,'html.parser')
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(response))
    return doc

In [48]:
def multiple_pages(n):
    url="https://books.toscrape.com/catalogue/page-"
    titles,prices,availability,urls=[],[],[],[]
    for page in range(1,n+1):
        doc=get_con(url+str(page)+'.html')
        titles.extend(get_book_titles(doc))
        prices.extend(get_book_price(doc))
        availability.extend(get_availability(doc))
        urls.extend(Book_link(doc.find_all('h3')))
        
    book_dic={
        'TITLE':titles,
        'PRICE':prices,
        'STOCK AVAILABILTY':availability,
        'URL':urls
    }
    return pd.DataFrame(book_dic)

In [49]:
multiple_pages()

,TITLE,PRICE,STOCK AVAILABILTY,URL
0,A Light in the ...,£51.77,In stock,https://books.toscrape.com/a-light-in-the-atti...
1,Tipping the Velvet,£53.74,In stock,https://books.toscrape.com/tipping-the-velvet_...
2,Soumission,£50.10,In stock,https://books.toscrape.com/soumission_998/inde...
3,Sharp Objects,£47.82,In stock,https://books.toscrape.com/sharp-objects_997/i...
4,Sapiens: A Brief History ...,£54.23,In stock,https://books.toscrape.com/sapiens-a-brief-his...
5,The Requiem Red,£22.65,In stock,https://books.toscrape.com/the-requiem-red_995...
6,The Dirty Little Secrets ...,£33.34,In stock,https://books.toscrape.com/the-dirty-little-se...
7,The Coming Woman: A ...,£17.93,In stock,https://books.toscrape.com/the-coming-woman-a-...
8,The Boys in the ...,£22.60,In stock,https://books.toscrape.com/the-boys-in-the-boa...
9,The Black Maria,£52.15,In stock,https://books.toscrape.com/the-black-maria_991...


In [51]:
multiple_pages(2).to_csv('SCB.csv',index = None)